In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
from fenics import *
from multiphenics import *
import numpy as np
import yaml
import pyvista as pv
from pathlib import Path
import imageio
from braininversion.PostProcessing import (extract_data,
                                           plot_partial_3D,
                                           load_results_and_mesh)
import warnings
warnings.filterwarnings("ignore")
#pv.set_plot_theme("document")


In [ ]:
cd ~/Documents/brain-force-inversion/brain-inversion/

In [ ]:
try:
    mesh_file = snakemake.input["subdomain_file"]
    sim_file = snakemake.input["sim_results"]
    sim_config_file = snakemake.input["sim_config_file"]
except NameError:
    mesh_name = "MRIExampleSegmentation_Nvcoarse"
    sim_name = "standard"

mesh_grid, sim_config, mesh_config, sim_file, source_expr =  load_results_and_mesh(mesh_name, sim_name)
cpos_close = [ [0.2, 0.15, -0.01], [0, 0, 0], [0.0, 0.0, 1.0] ]
ventricular_system = [dom["name"] for dom in mesh_config["domains"] if dom["name"] not in ["csf","parenchyma"]]
probes = mesh_config["probes"]
flatprobes = dict(**probes["sas"],**probes["parenchyma"],**probes["ventricular_system"])
domains = mesh_config["domains"]
name_to_label = {dom["name"]:dom["id"] for dom in domains}

In [ ]:
vent = extract_data(mesh_grid, "subdomains", ventricular_system, 5, sim_file, mesh_config)
csf = extract_data(mesh_grid, "subdomains", ["csf"], 5, sim_file, mesh_config)
par = extract_data(mesh_grid, "subdomains", ["parenchyma"], 5, sim_file, mesh_config)


csf_par_probes = flatprobes = dict(**probes["sas"],**probes["parenchyma"])

cpos_far = [(0.3, 0.2, -0.00), (-0.00, 0.000, -0.01), (0.0, 0.0, 1.0)]

p = pv.Plotter(notebook=True, polygon_smoothing=True)
#p.set_background("grey")
p.add_mesh(par, opacity=0.2, color="red",)
p.add_mesh(csf, opacity=0.2, color="white")
p.add_mesh(vent, opacity=0.6, color="blue")

p.add_point_labels(list(csf_par_probes.values()), csf_par_probes.keys(),
                   point_color="red", point_size=20,
                   render_points_as_spheres=False)
p.camera_position = cpos_far
p.screenshot(transparent_background=False, filename="results/par_points.png")
p.show()

In [ ]:
vent = extract_data(mesh_grid, "subdomains", ventricular_system, 5, sim_file, mesh_config)

cpos_close = [(0.2, 0.1, -0.00), (-0.00, 0.000, -0.01), (0.0, 0.0, 1.0)]

p = pv.Plotter(notebook=True, line_smoothing=True)
#p.set_background("grey")
p.add_mesh(vent, opacity=0.8, cmap="coolwarm", show_scalar_bar=False)

p.add_point_labels(list(probes["ventricular_system"].values()), probes["ventricular_system"].keys(),
                   point_color="red", point_size=20,
                   render_points_as_spheres=False)
p.camera_position = cpos_close
p.screenshot(transparent_background=False, return_img=False, window_size=None, filename="results/ventricle_points.png")
p.show()

In [ ]:
scenes = [{"var":"d", "mesh_parts": ["parenchyma"], "vec_scale":20, "clip":[(1,0,0)],
                                 "interactive" : {"color":"red", "opacity":1},
                                 "static" : {"color":"red", "opacity":1} },
                               {"var":"pF", "mesh_parts": ventricular_system,
                                 "interactive" : { "opacity":0.3},
                                 "static" : {"color":"red", "opacity":1} }]

p,_ = plot_partial_3D(mesh_grid, idx, scenes, sim_file, mesh_config, cpos_close, interactive=True)
p.show()

In [ ]:
cpos_far = [(0.3, 0.2, -0.05), (-0.00, 0.000, -0.02), (0.0, 0.0, 1.0)]
sargs = dict(title_font_size=20,label_font_size=16,shadow=True,n_labels=3,
             italic=True,font_family="arial", height=0.4, vertical=True, position_y=0.05)
sargs_u = None
sargs_pF = None

p_clim = [-20,20]

scenes = [{"var":"u", "mesh_parts": ["csf"], "vec_scale":1, "clip":[(1,0,0)],"arrow":True,
            "interactive" : {"color":"blue", "opacity":1},
            "static" : {"cmap":"speed", "opacity":1, "stitle":" mag(u)", "scalar_bar_args":sargs_u}},
           {"var":"pF", "mesh_parts": ventricular_system,
             "interactive" : {"color":"white", "opacity":0.5},
             "static" : {"cmap":"coolwarm", "opacity":0.8,
                        "scalar_bar_args":sargs_pF, "clim":p_clim}}]

p,_ = plot_partial_3D(mesh_grid, idx, scenes, sim_file, mesh_config, cpos_close, interactive=True)
p.show()

In [ ]:
scenes = [{"var":"d", "mesh_parts": ["parenchyma"], "warp":True, "clip":[(1,0,0)],
                                      "warp_fac":1e1, "vec_scale":1e1,
                                     "interactive" : {"color":"blue", "opacity":1},
                                     "static" : {"cmap":cm.speed, "opacity":1, "stitle":" mag(d)",
                                     "scalar_bar_args":sargs_u}}],

In [ ]:
sargs_u = None
sargs_pF = None
sargs_phi = None
func = lambda idx, clip, cpos, clim: plot_partial_3D(mesh_grid, idx,
                        [{"var":"phi", "mesh_parts": ["parenchyma"], "clip":clip,
                         "static" : {"cmap":"coolwarm","scalar_bar_args":sargs_phi, "clim":clim}},
                         {"var":"d", "mesh_parts": ["parenchyma"], "clip":clip, "arrow":True,"vec_scale":20,
                         "static" : {"color":"green","scalar_bar_args":sargs_phi}},
                        {"var":"pF", "mesh_parts": ventricular_system + ["csf"], "clip":clip,
                         "static" : {"cmap":"coolwarm", "scalar_bar_args":sargs_pF, "clim":clim}},
                        {"var":"u", "mesh_parts": ventricular_system + ["csf"], "clip":clip,
                         "arrow":True,"vec_scale":2,
                         "static" : {"color":"red", "scalar_bar_args":sargs_pF}}],
                         sim_file, mesh_config, cpos_close, interactive=False)

In [ ]:
i = 1
#indices = [2,4,6,8,10,12,14,16,18]
indices = [5,10]

origin = (0,0,0.001)
clips = ["y","x","z"]
dist = 0.4
cpos = [[(0, dist, 0), (0, 0, 0), (0, 0, 1)],
        [(dist,0, 0), (0, 0, 0), (0, 0, 1)],
        [(0, 0, dist*1.3), (0, 0, 0), (0, 1, 0)]
        ]
nind = len(indices)
size = 20
idx = 10
clim = None
p,_ = func(idx, {"normal":clips[i], "origin":origin}, cpos[i], clim)
p.show()

In [ ]:
%%capture

#max_phi = compute_glob_stat(max, mesh_grid, "phi", ["parenchyma"], range(num_steps))
##max_pF = compute_glob_stat(max, mesh_grid,"pF", ventricular_system + ["csf"], range(num_steps))
#min_phi = compute_glob_stat(max, mesh_grid, "phi", ["parenchyma"], range(num_steps))
#min_pF = compute_glob_stat(max, mesh_grid,"pF", ventricular_system + ["csf"], range(num_steps))


fig, axes = plt.subplots(nind, 3, figsize=(3*size, size*nind))
for j, idx in enumerate(indices):
    clim=None
    for i,c in enumerate(clips):
        #max_pF = compute_stat(max, mesh_grid, "pF", ventricular_system + ["csf"], idx)
        #min_pF = compute_stat(min, mesh_grid, "pF", ventricular_system + ["csf"], idx)
        #max_phi = compute_stat(max, mesh_grid, "phi", ["parenchyma"], idx)
        #min_phi = compute_stat(min, mesh_grid, "phi", ["parenchyma"], idx)
        #clim = (min(min_pF, min_phi), max(max_pF, max_phi))
        #clim=(-20,20)
        p, clim1 = func(idx, {"normal":c, "origin":origin}, cpos[i], clim)
        if i==0:
            clim=clim1
        p.show()
        img = p.screenshot(transparent_background=True, return_img=True, window_size=None)
        axes[j,i].imshow(img)
        axes[j,i].set_title(f"t = {times[idx]}")
plt.savefig("plots/pressure_evolution.pdf")

In [ ]:
vent = extract_data(mesh_grid, "u", ventricular_system + ["csf"], 5, sim_file, mesh_config)


In [ ]:
vent = extract_data(mesh_grid, "u", ventricular_system + ["csf"], 100, sim_file, mesh_config)
p = pv.Plotter()
vent["logu"] = np.log10(np.linalg.norm(vent["u"], axis=1) +1 )
arrow = vent.glyph(orient="logu", scale="logu", factor=4)
#arrow = vent.glyph(orient="u", scale="u", factor=2)
p.add_mesh(arrow)
p.show()

In [ ]:
np.log10(np.abs(vent["u"]))


In [ ]:
a = np.linspace(0,10,100)
np.log(a + 1)